In [ ]:
class Doctor:
    def __init__(self, name, specialty):
        self.name = name
        self.specialty = specialty
        self.slots = {}  # {"9:00-9:30": None (Available) or "Patient Name"}

    def add_slot(self, slot):
        self.slots[slot] = None

    def book_slot(self, slot, patient_name):
        if self.slots.get(slot) is None:
            self.slots[slot] = patient_name
            return True
        return False

    def cancel_slot(self, slot):
        if self.slots.get(slot) is not None:
            self.slots[slot] = None
            return True
        return False

    def get_available_slots(self):
        return [slot for slot, patient in self.slots.items() if patient is None]


class Patient:
    def __init__(self, name):
        self.name = name
        self.booked_slots = {}  # {"Doctor Name": "9:00-9:30"}

    def add_booking(self, doctor_name, slot):
        self.booked_slots[doctor_name] = slot

    def cancel_booking(self, doctor_name):
        if doctor_name in self.booked_slots:
            del self.booked_slots[doctor_name]


class AppointmentSystem:
    def __init__(self):
        self.doctors = {}  # {"Doctor Name": Doctor Object}
        self.patients = {}  # {"Patient Name": Patient Object}
        self.waitlist = {}  # {"Doctor Name": {"Slot": ["Patient Names"]}}

    def register_doctor(self, name, specialty):
        if name not in self.doctors:
            self.doctors[name] = Doctor(name, specialty)
            self.waitlist[name] = {}

    def register_patient(self, name):
        if name not in self.patients:
            self.patients[name] = Patient(name)

    def add_slot_to_doctor(self, doctor_name, slot):
        if doctor_name in self.doctors:
            self.doctors[doctor_name].add_slot(slot)

    def book_appointment(self, patient_name, doctor_name, slot):
        if patient_name not in self.patients or doctor_name not in self.doctors:
            return "Invalid patient or doctor."

        doctor = self.doctors[doctor_name]
        patient = self.patients[patient_name]

        # Check if slot is already booked
        if doctor.slots.get(slot) is not None:
            # Add to waitlist
            if slot not in self.waitlist[doctor_name]:
                self.waitlist[doctor_name][slot] = []
            self.waitlist[doctor_name][slot].append(patient_name)
            return f"Slot already booked. {patient_name} added to waitlist."

        # Book the slot
        if doctor.book_slot(slot, patient_name):
            patient.add_booking(doctor_name, slot)
            return f"Slot {slot} booked with Dr. {doctor_name}."

    def cancel_appointment(self, patient_name, doctor_name):
        if patient_name not in self.patients or doctor_name not in self.doctors:
            return "Invalid patient or doctor."

        patient = self.patients[patient_name]
        doctor = self.doctors[doctor_name]

        # Get the slot the patient booked
        slot = patient.booked_slots.get(doctor_name)
        if slot is None:
            return "No booking found to cancel."

        # Cancel the slot
        if doctor.cancel_slot(slot):
            patient.cancel_booking(doctor_name)

            # Check if there are people on the waitlist
            if slot in self.waitlist[doctor_name] and self.waitlist[doctor_name][slot]:
                next_patient = self.waitlist[doctor_name][slot].pop(0)
                doctor.book_slot(slot, next_patient)
                self.patients[next_patient].add_booking(doctor_name, slot)
                return f"Appointment canceled. Slot reassigned to {next_patient}."

            return "Appointment canceled."

    def view_booked_appointments(self, name):
        if name in self.patients:
            return self.patients[name].booked_slots
        if name in self.doctors:
            doctor = self.doctors[name]
            return {slot: patient for slot, patient in doctor.slots.items() if patient is not None}
        return "Invalid name."


# Driver code
if __name__ == "__main__":
    system = AppointmentSystem()

    # Register doctors and patients
    system.register_doctor("Dr. Smith", "Cardiologist")
    system.register_doctor("Dr. Lee", "Dermatologist")
    system.register_patient("Alice")
    system.register_patient("Bob")

    # Add slots to doctors
    system.add_slot_to_doctor("Dr. Smith", "9:00-9:30")
    system.add_slot_to_doctor("Dr. Smith", "9:30-10:00")
    system.add_slot_to_doctor("Dr. Lee", "9:00-9:30")

    # Book appointments
    print(system.book_appointment("Alice", "Dr. Smith", "9:00-9:30"))  # Alice books Dr. Smith
    print(system.book_appointment("Bob", "Dr. Smith", "9:00-9:30"))    # Bob added to waitlist

    # View appointments
    print(system.view_booked_appointments("Alice"))  # Alice's appointments
    print(system.view_booked_appointments("Dr. Smith"))  # Dr. Smith's appointments

    # Cancel an appointment
    print(system.cancel_appointment("Alice", "Dr. Smith"))  # Alice cancels, Bob gets the slot

    # View updated appointments
    print(system.view_booked_appointments("Dr. Smith"))
